In [11]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(1234)

np.set_printoptions(formatter={'all':lambda x: '%.3f' % x})

from IPython.display import Image
from numpy.core.umath_tests import matrix_multiply as mm

from scipy.optimize import minimize
from scipy.stats import bernoulli, binom

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  


In [41]:
m = 10
theta_A = 0.8
theta_B = 0.3
theta_0 = [theta_A, theta_B]

coin_A = bernoulli(theta_A)
coin_B = bernoulli(theta_B)

xs = map(sum, [coin_A.rvs(m), coin_A.rvs(m), coin_B.rvs(m), coin_A.rvs(m), coin_B.rvs(m)])
zs = [0, 0, 1, 0, 1]


In [47]:
xs =  np.fromiter(xs, dtype=np.int)
xs

array([8.000, 9.000, 3.000, 7.000, 5.000])

# Exact Solution

In [43]:
ml_A = np.sum(xs[[0,1,3]])/(3.0*m)
ml_B = np.sum(xs[[2,4]])/(2.0*m)
ml_A, ml_B

(0.8, 0.4)

# Numerical Estimate

In [ ]:
def neg_loglik(thetas, n, xs, zs):
    return -np.sum([binom(n, thetas[z]).logpmf(x) for (x, z) in zip(xs, zs)])

In [44]:
bnds = [(0,1), (0,1)]
minimize(neg_loglik, [0.5, 0.5], args=(m, xs, zs),
         bounds=bnds, method='tnc', options={'maxiter': 100})

     fun: 7.25864589062827
     jac: array([-0.000, 0.000])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 23
     nit: 6
  status: 1
 success: True
       x: array([0.800, 0.400])

In [81]:
xs = np.array([(5,5), (9,1), (8,2), (4,6), (7,3)])
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])

tol = 0.01
max_iter = 100

ll_old = 0
for i in range(max_iter):
    ws_A = []
    ws_B = []

    vs_A = []
    vs_B = []

    ll_new = 0

    # E-step: calculate probability distributions over possible completions
    for x in xs:
        #print(x)
        # multinomial (binomial) log likelihood
        ll_A = np.sum([x*np.log(thetas[0])])
        ll_B = np.sum([x*np.log(thetas[1])])
        #print(ll_A)
        #print(ll_B)

        # [EQN 1]
        denom = np.exp(ll_A) + np.exp(ll_B)
        w_A = np.exp(ll_A)/denom
        w_B = np.exp(ll_B)/denom

        ws_A.append(w_A)
        ws_B.append(w_B)

        # used for calculating theta
        vs_A.append(np.dot(w_A, x))
        vs_B.append(np.dot(w_B, x))
        #print(vs_A)

        # update complete log likelihood
        ll_new += w_A * ll_A + w_B * ll_B

    # M-step: update values for parameters given current distribution
    # [EQN 2]
    thetas[0] = np.sum(vs_A, 0)/np.sum(vs_A)
    thetas[1] = np.sum(vs_B, 0)/np.sum(vs_B)
    # print distribution of z for each x and current parameter estimate

    print("Iteration: {}".format(i+1))
    print("theta_A = {}, theta_B = {}, ll = {}".format(thetas[0,0], thetas[1,0], ll_new))
    if np.abs(ll_new - ll_old) < tol:
        break
    ll_old = ll_new

[array([2.246, 2.246])]
[array([2.246, 2.246]), array([7.245, 0.805])]
[array([2.246, 2.246]), array([7.245, 0.805]), array([5.868, 1.467])]
[array([2.246, 2.246]), array([7.245, 0.805]), array([5.868, 1.467]), array([1.409, 2.113])]
[array([2.246, 2.246]), array([7.245, 0.805]), array([5.868, 1.467]), array([1.409, 2.113]), array([4.531, 1.942])]
[array([1.479, 1.479])]
[array([1.479, 1.479]), array([7.304, 0.812])]
[array([1.479, 1.479]), array([7.304, 0.812]), array([5.651, 1.413])]
[array([1.479, 1.479]), array([7.304, 0.812]), array([5.651, 1.413]), array([0.761, 1.141])]
[array([1.479, 1.479]), array([7.304, 0.812]), array([5.651, 1.413]), array([0.761, 1.141]), array([4.015, 1.721])]
[array([1.088, 1.088])]
[array([1.088, 1.088]), array([7.829, 0.870])]
[array([1.088, 1.088]), array([7.829, 0.870]), array([6.009, 1.502])]
[array([1.088, 1.088]), array([7.829, 0.870]), array([6.009, 1.502]), array([0.446, 0.670])]
[array([1.088, 1.088]), array([7.829, 0.870]), array([6.009, 1.502